# Description

This notebook is used to run an pre-trained ML model to predict whether irrigation is in use in a geographic zone for a date range.  It generates images which can then be visualized in Google Earth Engine.

The SEBAL calculations for ET are leveraged from https://github.com/gee-hydro/geeSEBAL


# Setup Notebook

In [1]:
#!pip install geemap
#!pip install geopandas

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project='second-impact-342800')
import geemap

import numpy as np
import pandas as pd
import datetime
import json
from timeit import default_timer as timer
from datetime import timedelta
import ipyleaflet
import ipywidgets as widgets
from tqdm.notebook import tqdm
import calendar

Enter verification code: 4/1AfJohXkx1jz4QjW7hnO-FIr2CoHGkExEiKEsZ__QKHiE2HOsGeT-u0uHWrY

Successfully saved authorization token.


In [3]:
# Run in a cloud environment
#%cd /content
#!git clone https://github.com/gee-hydro/geeSEBAL.git
#!git clone https://github.com/earthdataanalytics/irrigation.git
#!cp -r geeSEBAL/etbrasil/ irrigation/
#%cd irrigation/

/content
fatal: destination path 'irrigation' already exists and is not an empty directory.
/content/irrigation


#### Custom libraries

In [2]:
from etbrasil.geesebal import Image_bcj
from etbrasil.geesebal import etretrieval as etr
from pipeline import boundaries as bnd
from pipeline import cropmasks as msk
from pipeline import labeledsamples as ls
from pipeline import dataextract as dx
from pipeline import utils

# Train GEE Random Forest classifier

##### Set training parameters

In [5]:
# To customize for each run
project_name = 'Restart'
startDate = '2021-08-01'
endDate = '2022-10-31'
model_name = '20230925_gee_cost_est' # This retrieves the best model for the area.  Was previously f'{project_name}'

# Defaults
et_var = 'ET_24h_R'
label_var = 'loc_type'
cols = [et_var, 'NDVI', 'LandT_G', 'last_rain', 'sum_precip_priorX',	'mm', label_var, 'latitude', 'longitude']

##### Retrieve training data

In [6]:
model_path = f'../../runs/{model_name}/best/'
with open(model_path+'summary_stats.json', 'r', encoding='utf-8') as f:
    local_best = json.load(f)

best_run = local_best['exp_ref']
feature_path = f'../../runs/{model_name}/{best_run}/'

In [7]:
with ee.profilePrinting():
    ee.data.setDefaultWorkloadTag('hydracarta_backend')
    ee.data.setWorkloadTag('hydracarta_train_model')

    df_features = pd.read_pickle(feature_path+'features.pkl')
    df_features = df_features[cols]

    # set all columns to numeric
    for col in cols:
        df_features[col] = df_features[col].astype(float)

    # convert from celsius to kelvin
    df_features.LandT_G = df_features.LandT_G + 273.15

    fc_data = geemap.pandas_to_ee(df_features, latitude="latitude", longitude="longitude")
    print('Total number of samples:   ', fc_data.size().getInfo())

    fc_split = utils.train_test_split(fc_data)
    print('Number of training samples:', fc_split['train_partition'].size().getInfo())

Total number of samples:    2644
Number of training samples: 2092


##### Create and train GEE RandomForest Classifier

In [8]:
with ee.profilePrinting():
    ee.data.setWorkloadTag('hydracarta_train_model')

    num_trees = 1000
    bag_fraction = 0.63
    #variables_per_split = 10
    feature_list = cols[:6]

    classifier = ee.Classifier.smileRandomForest(
        numberOfTrees=num_trees,
    #    variablesPerSplit=variables_per_split,
        bagFraction=bag_fraction,
        seed=10
    )
    classifier = classifier.train(
        features=fc_split['train_partition'],
        classProperty=label_var,
        inputProperties=feature_list,
        subsamplingSeed=10
    )

#### Score GEE classifier

In [9]:
%%time
with ee.profilePrinting():
    ee.data.setDefaultWorkloadTag('hydracarta_backend')
    ee.data.setWorkloadTag('hydracarta_train_model')

    validated = fc_split['test_partition'].classify(classifier)

    # Get a confusion matrix representing expected accuracy.
    if classifier.mode() != 'PROBABILITY':
        validation_matrix = validated.errorMatrix(label_var, 'classification')
        cm = validation_matrix.getInfo()
        gee_acc = validation_matrix.accuracy().getInfo()
        gee_kappa = validation_matrix.kappa().getInfo()

        print(f'Validation error matrix: {cm}')
        print(f'Validation accuracy:     {gee_acc:.3f}')
        print(f'Validation kappa:        {gee_kappa:.3f}')

Validation error matrix: [[150, 95], [71, 236]]
Validation accuracy:     0.699
Validation kappa:        0.385
CPU times: user 2.45 s, sys: 64.9 ms, total: 2.52 s
Wall time: 9.46 s


#### Compare GEE and local classfiers

In [10]:
print(f'GEE RF Classifier Accuracy   {gee_acc:.2f}')
print(f'Local RF Classifier Accuracy {local_best["rf_accuracy"]:.2f}')

GEE RF Classifier Accuracy   0.70
Local RF Classifier Accuracy 0.68


# Infer classes for AOI

##### Define Area of Interest (AOI)

In [11]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.centerObject(bnd.central_ca)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [12]:
# In general it has been best to select an AOI which is less than (<) 3,000 hectares in area
# otherwise the export step tends to crash when processing takes too long, which occurs
# frequently when area > 3k hectares.

if Map and (len(Map.draw_features) > 0):
    aoi = ee.FeatureCollection(Map.draw_features)
else:
    print("Please select an Area of Interest on the map")

In [13]:
junk = Image_bcj(window_start='2021-08-01', window_end='2021-08-31', 
                      aoi=aoi, cloud_max=30, ls_col2=True,
                      et_var=et_var) \
                .ETandMeteo
junk.first().sample(numPixels=5).getInfo()

EEException: ignored

In [14]:
junk.propertyNames().getInfo()

['type_name',
 'keywords',
 'visualization_1_bands',
 'thumb',
 'visualization_1_max',
 'description',
 'source_tags',
 'visualization_1_name',
 'system:id',
 'visualization_0_max',
 'title',
 'product_tags',
 'provider',
 'visualization_1_min',
 'visualization_0_min',
 'system:version',
 'visualization_0_name',
 'date_range',
 'visualization_2_bands',
 'visualization_2_name',
 'period',
 'visualization_2_min',
 'provider_url',
 'sample',
 'tags',
 'visualization_2_max',
 'visualization_0_bands']

In [19]:
junk.first().bandNames().getInfo()

['UB',
 'B',
 'GR',
 'R',
 'NIR',
 'SWIR_1',
 'SWIR_2',
 'BRT',
 'pixel_qa',
 'ALFA',
 'Rn24h_G',
 'AirT_G',
 'RH_G',
 'ux_G',
 'SW_Down',
 'NDVI',
 'EVI',
 'SAVI',
 'T_LST',
 'LAI',
 'e_0',
 'e_NB',
 'longitude',
 'latitude',
 'NDVI_neg',
 'pos_NDVI',
 'int',
 'sd_ndvi',
 'NDWI',
 'BRT_R',
 'T_LST_DEM',
 'LST_neg',
 'LST_NW',
 'Rl_up',
 'Rs_down',
 'Tao_sw',
 'ES',
 'EA']

##### Retrieve data for inference

In [14]:
def retrieveRGBandClassPredictions(startDate=None, endDate=None, applyCropMask=False):
    if (startDate==None) or (endDate==None):
        print("Please specify start and end date")
        return None, None

    # changed ls_col2 from False to True on 2023-02-07.  I had updated the code
    # in Image_bcj to handle LS Collection 2, but was not yet using it.
    # Noticed that no images after 2021 were retrieved, due to Col2 superceding Col1.
    etcol = Image_bcj(window_start=startDate, window_end=endDate, 
                      aoi=aoi, cloud_max=30, ls_col2=True,
                      et_var=et_var) \
                .ETandMeteo

    if applyCropMask: # added on 2023-02-07
        etcol = etcol.map(lambda x: x.updateMask(msk.createGFSADmask(aoi)))

    num_images = etcol.size().getInfo()
    print('Number of ET sample images retrieved: ', num_images)

#    rgbCol = ['R', 'G', 'B']
#    rgb_mosaic = dx.buildImageCollection(aoi=aoi, start=startDate, end=endDate, ls_all=False) \
#                      .mosaic() \
#                      .clip(aoi) \
#                      .select(['B4', 'B3', 'B2']) \
#                      .rename(rgbCol) \
#                      .set('custom:date', startDate+'_'+endDate)

    rgbCol = ['R', 'G', 'B']
    # 2022.08 - in versions before the monthly data labels version Aug-2022, 
    # the rgb_mosaic was calculated as a mosaic, however this started producing
    # too many bad images.
    rgb_mosaic = etcol \
                      .first() \
                      .clip(aoi) \
                      .select(['R', 'GR', 'B']) \
                      .rename(rgbCol) \
                      .set('custom:date', startDate+'_'+endDate)

    pred_col = etcol.map(lambda x: x.classify(classifier))

    # create a map which takes the mean over all predictions in the time window
    # to represent the probability of the pixel being irrigated or not during the window
    pred_prob = pred_col.mean() \
                        .clip(aoi) \
                        .set('custom:date', startDate+'_'+endDate) \
                        .set('custom:num_input_images', num_images)
            
    return rgb_mosaic, pred_prob, etcol, num_images

In [15]:
%%time
rgb_mosaic, pred_prob, et_data, cnt = retrieveRGBandClassPredictions(startDate, endDate, applyCropMask=False)

Number of ET sample images retrieved:  22
CPU times: user 745 ms, sys: 18.4 ms, total: 763 ms
Wall time: 912 ms



##### Visualize results


In [16]:
rgbCol = ['R', 'G', 'B']
rgb_viz = {'bands': rgbCol, 'min': 0.0, 'max': 3000, 'gamma': 1.4} # LS-Col-1-SR
pred_viz = {'min': 0.0, 'max': 1.0, 'palette': ['red', 'blue']}
prob_viz = {'min': 0.0, 'max': 1.0, 'palette': ['red', 'magenta', 'pink', 'yellow', 'aqua', 'blue', 'darkblue']}

if False: # skip this cell 
    Map.centerObject(aoi)
    Map.addLayer(aoi, {}, 'aoi boundary', True, 0.55)
    Map.addLayer(rgb_mosaic, rgb_viz, 'rgb', True, 1.0)
    #Map.addLayer(pred_mosaic, pred_viz, 'predictions', False, 0.5)
    Map.addLayer(pred_prob, prob_viz, 'probabilities', True, 1.0)
    Map

# Export Data

In [17]:
def exportImages(rgb_mosaic, pred_prob, startDate, endDate, aoi=None):
    #print("RGB export start")
    rgb_filename = f'../outputmaps/monthly_rgb_{startDate}_{endDate}.tif'
    geemap.ee_export_image(rgb_mosaic, filename=rgb_filename, scale=30, 
                            region=aoi, file_per_band=False,
                            timeout=300)
    #print("RGB export end")
    
    #print("Class prediction export start")
    class_filename = f'../outputmaps/monthly_classes_{startDate}_{endDate}.tif'
    geemap.ee_export_image(pred_prob, filename=class_filename, scale=30, 
                       region=aoi, file_per_band=False,
                       timeout=1200)
    #print("Class prediction export end")

In [18]:
%%time
from dateutil.relativedelta import relativedelta
# The output from this cell is saved to the local instance under the folder /outputmaps
# No tasks are created on GEE

retrieval_dates = pd.date_range(startDate,endDate, 
              freq='MS').strftime("%Y-%m-%d").tolist()

for start_date in tqdm(retrieval_dates):
    start_date_ts = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date = start_date_ts + relativedelta(months=1) + timedelta(days=-1)
    end_date = end_date.strftime('%Y-%m-%d')

    print('Processing', start_date, end_date)
    rgb_mosaic, pred_prob, et_data, img_cnt = retrieveRGBandClassPredictions(start_date, end_date)
    if img_cnt > 1:
        exportImages(rgb_mosaic, pred_prob, start_date, end_date, aoi=aoi.geometry()) # or aoi=Map.user_roi
    else:
        print('\tWarning:  Less than two(2) ET images retrieved for period', start_date, end_date)

  0%|          | 0/15 [00:00<?, ?it/s]

Processing 2021-08-01 2021-08-31
Number of ET sample images retrieved:  2
Generating URL ...
An error occurred while downloading.
Collection.first: Error in map(ID=LC08_043034_20210821):
Image.constant: Max (NaN) cannot be less than min (NaN).
Generating URL ...
An error occurred while downloading.
reduce.mean: Error in map(ID=LC08_043034_20210821):
Image.constant: Max (NaN) cannot be less than min (NaN).
Processing 2021-09-01 2021-09-30
Number of ET sample images retrieved:  2
Generating URL ...


KeyboardInterrupt: ignored

In [165]:
%cd /content/outputmaps

/content/outputmaps


In [166]:
import subprocess
#!gsutil mv *.tif gs://staging_demo/CA_monthly_01
run_string = f'gsutil mv *.tif gs://staging_demo/{project_name}'
subprocess.call(run_string, shell=True)

0

Create image collections for each type

In [171]:
run_string = f'earthengine create collection projects/eda-bjonesneu-proto/assets/irrigation/monthly_classes_{project_name}'
subprocess.call(run_string, shell=True)

0

In [172]:
run_string = f'earthengine create collection projects/eda-bjonesneu-proto/assets/irrigation/monthly_rgb_{project_name}'
subprocess.call(run_string, shell=True)

0

In [173]:
!earthengine ls projects/eda-bjonesneu-proto/assets/irrigation

projects/eda-bjonesneu-proto/assets/irrigation/classes_col
projects/eda-bjonesneu-proto/assets/irrigation/labels
projects/eda-bjonesneu-proto/assets/irrigation/labels_new
projects/eda-bjonesneu-proto/assets/irrigation/labels_old
projects/eda-bjonesneu-proto/assets/irrigation/monthly_classes_Clark_Ranch
projects/eda-bjonesneu-proto/assets/irrigation/monthly_classes_Westwind
projects/eda-bjonesneu-proto/assets/irrigation/monthly_classes_col_monthly
projects/eda-bjonesneu-proto/assets/irrigation/monthly_classes_col_monthly_02
projects/eda-bjonesneu-proto/assets/irrigation/monthly_rgb_Clark_Ranch
projects/eda-bjonesneu-proto/assets/irrigation/monthly_rgb_Westwind
projects/eda-bjonesneu-proto/assets/irrigation/monthly_rgb_col_monthly
projects/eda-bjonesneu-proto/assets/irrigation/monthly_rgb_col_monthly_02
projects/eda-bjonesneu-proto/assets/irrigation/rgb_col


Upload the individual images to each image collection

In [174]:
from google.cloud import storage
import subprocess

bucket_name = "staging_demo"
storage_client = storage.Client.from_service_account_json("/content/irrigation/second-impact-342800-51af159903ca.json")
blobs = storage_client.list_blobs(bucket_name)

cnt=0
for blob in blobs:
    if project_name in blob.name:
        filename = blob.name
        if '.tif' in filename:
            tmp_name = filename.split('/')[-1]
            project_type, asset_type, start, end = tmp_name[:-4].split('_')
            asset_name = f'projects/eda-bjonesneu-proto/assets/irrigation/{project_type}_{asset_type}_{project_name}/{tmp_name[:-4]}'
            gs_name = f'gs://{bucket_name}/{filename}'
            run_string = f"earthengine upload image --asset_id={asset_name} {gs_name}"
            #print(run_string)
            out = subprocess.call(run_string, shell=True)

In [ ]:
STOP

##### Utility to delete unneeded files in folders

In [ ]:
#!earthengine ls projects/eda-bjonesneu-proto/assets/irrigation/rgb > todelete.txt

In [ ]:
if False:
    with open('todelete.txt') as todelete:
        for item in todelete:
            run_string = f"earthengine rm {item}"
            out = subprocess.call(run_string, shell=True)

In [ ]:
#!earthengine ls projects/eda-bjonesneu-proto/assets/irrigation/rgb

Asset "projects/eda-bjonesneu-proto/assets/irrigation/classes" not found.
